In [1]:
#csvファイルの読み込み
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df_train = pd.read_csv(r"C:\Users\eikik\python\python_dat\equity-post-HCT-survival-predictions\train_preprocessed.csv")

In [2]:
#データの確認
df_train

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,target
0,N/A - non-malignant indication,No,Poor,No,-0.001785,-0.002142,No TBI,No,0.754491,Bone marrow,...,No,Permissive mismatched,Unrelated,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.834709
1,Intermediate,No,Intermediate,No,0.595336,0.804529,"TBI +- Other, >cGy",No,0.754491,Peripheral blood,...,No,Permissive mismatched,Related,"N/A, Mel not given",0.751129,No,0.675826,Yes,0.755836,1.367042
2,N/A - non-malignant indication,No,Poor,No,0.595336,0.804529,No TBI,No,0.754491,Bone marrow,...,No,Permissive mismatched,Related,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.813559
3,High,No,Intermediate,No,0.595336,0.804529,No TBI,No,0.754491,Bone marrow,...,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.846174
4,High,No,Poor,No,0.595336,0.804529,No TBI,No,0.754491,Peripheral blood,...,No,Permissive mismatched,Related,MEL,0.751129,No,0.675826,No,0.755836,-0.800830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Intermediate - TED AML case <missing cytogenetics,No,Favorable,No,0.595336,0.804529,No TBI,No,0.754491,Peripheral blood,...,No,Bi-directional non-permissive,Related,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.811172
28796,High,No,Poor,Yes,-1.931355,-2.058671,No TBI,No,-0.124932,Peripheral blood,...,No,GvH non-permissive,Related,"N/A, Mel not given",-0.616506,Yes,-1.630964,Yes,-0.389694,1.241045
28797,TBD cytogenetics,No,Poor,No,0.595336,0.804529,No TBI,No,0.754491,Peripheral blood,...,No,GvH non-permissive,Unrelated,"N/A, Mel not given",0.751129,No,0.675826,No,0.755836,-0.820817
28798,N/A - non-malignant indication,No,Poor,No,-1.931355,-2.058671,No TBI,No,-1.883777,Peripheral blood,...,No,Permissive mismatched,Related,MEL,-1.984141,No,-1.630964,No,-2.107990,-0.836307


In [6]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold

# Custom function to compute the Stratified Concordance Index (C-index)
def stratified_c_index(y_true, y_pred, groups):
    unique_groups = np.unique(groups)
    c_indices = []

    for group in unique_groups:
        mask = groups == group
        if sum(mask) > 1:  
            y_true_group = y_true[mask]
            y_pred_group = y_pred[mask]
            concordant = 0
            permissible = 0

            for i in range(len(y_true_group)):
                for j in range(i + 1, len(y_true_group)):
                    if y_true_group[i] != y_true_group[j]:
                        permissible += 1
                        if (y_pred_group[i] > y_pred_group[j] and y_true_group[i] > y_true_group[j]) or \
                           (y_pred_group[i] < y_pred_group[j] and y_true_group[i] < y_true_group[j]):
                            concordant += 1

            c_indices.append(concordant / permissible if permissible > 0 else 0)

    c_indices = np.array(c_indices)
    return np.mean(c_indices) - np.std(c_indices)

In [7]:
# Sample dataset
X = df_train.drop(columns=['target'], axis=1)
y = df_train['target']
race_groups = df_train['race_group']  

# Convert categorical features for CatBoost
cat_features = list(X.select_dtypes(include=['object', 'category']).columns)

# K-Fold Cross Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
final_predictions = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, race_groups)):
    print(f"Fold {fold + 1}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    race_val = race_groups.iloc[val_idx]

    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)

    # Model with tuned hyperparameters
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=3,
        loss_function='RMSE',
        random_seed=42,
        verbose=100,
        # task_type='GPU',
    )

    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50)

    # Predict and compute metric
    y_val_pred = model.predict(X_val)
    fold_score = stratified_c_index(y_val.values, y_val_pred, race_val.values)
    print(f"Stratified C-Index for Fold {fold + 1}: {fold_score}")

    final_predictions[val_idx] = y_val_pred

# Overall Stratified Concordance Index
overall_score = stratified_c_index(y.values, final_predictions, race_groups.values)
print(f"Overall Stratified C-Index: {overall_score}")

Fold 1
0:	learn: 0.9931946	test: 1.0011128	best: 1.0011128 (0)	total: 265ms	remaining: 4m 24s
100:	learn: 0.9034413	test: 0.9215736	best: 0.9215736 (100)	total: 8.92s	remaining: 1m 19s
200:	learn: 0.8864054	test: 0.9127396	best: 0.9127396 (200)	total: 16.5s	remaining: 1m 5s
300:	learn: 0.8727120	test: 0.9070134	best: 0.9070134 (300)	total: 23.7s	remaining: 55s
400:	learn: 0.8636249	test: 0.9042480	best: 0.9042480 (400)	total: 31.3s	remaining: 46.8s
500:	learn: 0.8556466	test: 0.9025682	best: 0.9025473 (495)	total: 39.4s	remaining: 39.3s
600:	learn: 0.8494742	test: 0.9015785	best: 0.9014578 (584)	total: 46.8s	remaining: 31.1s
700:	learn: 0.8438472	test: 0.9008659	best: 0.9008659 (700)	total: 54.3s	remaining: 23.2s
800:	learn: 0.8385085	test: 0.9000712	best: 0.9000584 (797)	total: 1m 1s	remaining: 15.4s
900:	learn: 0.8336595	test: 0.8997416	best: 0.8997370 (898)	total: 1m 10s	remaining: 7.72s
999:	learn: 0.8293791	test: 0.8993012	best: 0.8992766 (994)	total: 1m 17s	remaining: 0us

bestTe